In [2]:
"""임베딩 벡터에 토큰의 상대적/절대적 위치에 대한 정보를 추가하는 클래스. 

        sinusoidal를 이용해 임베딩 벡터에 위치 정보를 추가. 
        이때, log와 지수의 성질을 이용해 효율적으로 계산한다.
    
"""
import math
import pandas as pd
import torch
import torch.nn as nn
class PositionalEncoding(nn.Module):
    '''PositionalEncoding 클래스로, 임베딩 벡터에 위치 정보 추가. 

    Attributes:
        d_model(int): 임베딩 벡터의 차원.
        dropout(nn.Dropout): 드롭아웃.
        max_len(int): 최대 시퀀스 길이.
        pe(torch.Tensor): 위치 정보를 나타내는 (max_len, d_model) 크기의 행렬.
    '''
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        """PositionalEncoding 클래스의 생성자
        
        Args:
            d_model(int): 임베딩 벡터의 차원.
            dropout(float): 드롭아웃 확률.
            max_len(int): 최대 시퀀스 길이.
        """
        super(PositionalEncoding, self).__init__()
        
        self.pe = torch.zeros(max_len, d_model) # (max_len, d_model)크기의 행렬을 0으로 초기화 
        
        pos = torch.arange(0, max_len).unsqueeze(1).float() # 0~(max-len-1)까지 정수 생성 후 (max_len, 1)크기의 행렬 생성(브로드캐스팅을 위해)
        i_2 = torch.arange(0, d_model, 2).float() # 0부터 d_model까지 짝수만, 2i를 만든 것 (d_model//2, )
        div_term = torch.exp( i_2 * (-math.log(10000.0) / d_model)) # 로그와 지수의 성질을 이용해 10000^(2i/d_model) 계산
        
        self.pe[:, 0::2] = torch.sin(pos * div_term) # i가 짝수인 경우
        self.pe[:, 1::2] = torch.cos(pos * div_term) # i가 홀수인 경우
        self.pe = self.pe.unsqueeze(0) # batch를 위해 추가 (1, max_len, d_model) 크기를 가지게 됨.
        
        self.dropout = nn.Dropout(p=dropout) # 임베딩 벡터에 위치 정보를 추가한 후 dropout하기 위해

    def forward(self, x):
        """입력에 위치 인코딩 더하고 dropout 적용. 
        Args:
            x(torch.Tensor): 입력 텐서로, (배치 크기, 시퀀스 길이, 임베딩 차원)을 가짐. 

        Returns:
            torch.Tensor: (배치 크기, 시퀀스 길이, 임베딩 차원) 텐서에 dropout 추가. 
        """
        x = x + self.pe[:, :x.size(1), :] # 입력 시퀀스의 길이 맞게 위치 인코딩 추가
        return self.dropout(x)

In [16]:
# 이거는 http://nlp.seas.harvard.edu/annotated-transformer/ 여기서 아예 가져왔다. 
import altair as alt
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))

    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )

    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )



def show_example(fn, args=[]):
    if __name__ == "__main__":
        return fn(*args)

show_example(example_positional)

alt.Chart(...)